<a href="https://colab.research.google.com/github/choeuneheol/python-practice/blob/master/0903moviereview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install konlpy
!pip install tqdm
!pip install git+https://github.com/haven-jeon/PyKoSpacing
!pip install git+https://github.com/ssut/py-hanspell.git
# datasets (huggingface 자연어처리 dataset)
# konlpy (한글 자연어처리 Tool)
# tqdm (반복문의 Progress Bar)
# PyKoSpacing (한글 띄어쓰기 보정 Tool)
# Py-Hanspell(네이버 하늘 맞춤법 검사기 기반 한글 맞춤법 보정 Tool)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 28.5 MB/s 
     |████████████████████████████████| 120 kB 70.0 MB/s 
     |████████████████████████████████| 212 kB 68.5 MB/s 
     |████████████████████████████████| 115 kB 74.1 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 19.4 MB 19.1 MB/s 
     |████████████████████████████████| 453 kB 52.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/haven-jeo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-1bwrn5mk
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-1bwrn5mk
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4868 sha256=ae9ed6f864cc7c492095aac535f14deddb3a71dbc289898531fd28a07d869b83
  Stored in directory: /tmp/pip-ephem-wheel-cache-o75cy729/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


# 2-0. 데이터셋 준비
Naver Sentiment Movie Corpus(nsmc)
영화 리뷰 별점을 기준으로 긍정(1), 부정(0)으로 구분한 데이터셋

In [2]:
from datasets import load_dataset

#nsmc : 데이터셋 다운로드
dataset = load_dataset("nsmc")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/150000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset nsmc downloaded and prepared to /root/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# dataset의 구조
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


In [4]:
# 첫 5개의 문장을 추출
for i in range(5):
  print(dataset["train"][i])

nsmc5_sents = [dataset["train"][idx]["document"] for idx in range(5)]

{'id': '9976970', 'document': '아 더빙.. 진짜 짜증나네요 목소리', 'label': 0}
{'id': '3819312', 'document': '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나', 'label': 1}
{'id': '10265843', 'document': '너무재밓었다그래서보는것을추천한다', 'label': 0}
{'id': '9045019', 'document': '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정', 'label': 0}
{'id': '6483659', 'document': '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다', 'label': 1}


In [5]:
from konlpy.tag import Okt
okt = Okt()

# Okt 형태소 분석기로 문장을 형태소 단위의 token으로 분리
# 속도가 빠르다
# 정제되지 않은 데이터에서 좋은 성능을 보인다

for sent_i in nsmc5_sents:
  words_i = [w for w, p in okt.pos(sent_i)]
  print(sent_i)
  print(words_i)
  print()

아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증나네요', '목소리']

흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초딩', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

너무재밓었다그래서보는것을추천한다
['너', '무재', '밓었', '다그', '래서', '보는것을', '추천', '한', '다']

교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']



In [6]:
from hanspell import spell_checker
# 스펠체커를 통해서 맞춤법 고치고
from pykospacing import Spacing
# 스페싱이란 함수를 써서 띄어쓰기를 고침

spacing = Spacing()
tokenized_nsmc5_corpus = []

#한글 전처리 패키지인 hanspell과 pykospacing으로 전처리한 경우
for sent_i in nsmc5_sents:
  new_sent_i = spell_checker.check(sent_i).checked
  new_sent_i = spacing(new_sent_i)
  word_i = [w for w, p in okt.pos(new_sent_i)]

  tokenized_nsmc5_corpus.append(words_i)
  print("Before: {0}".format(sent_i))
  print("After: {0}".format(sent_i))
  print(word_i)
  print()
 # 전처리 완료

Before: 아 더빙.. 진짜 짜증나네요 목소리
After: 아 더빙.. 진짜 짜증나네요 목소리
['아', '더빙', '..', '진짜', '짜증', '나네요', '목소리']

Before: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
After: 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
['흠', '...', '포스터', '보고', '초등학생', '영화', '줄', '....', '오버', '연기', '조차', '가볍지', '않구나']

Before: 너무재밓었다그래서보는것을추천한다
After: 너무재밓었다그래서보는것을추천한다
['너무', '재밓었다', '그래서', '보는', '것', '을', '추천', '한다']

Before: 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
After: 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
['교도소', '이야기', '구먼', '..', '솔직히', '재미', '는', '없다', '..', '평점', '조정']

Before: 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
After: 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
['사이먼', '페그', '의', '익살스러운', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이', '뻐', '보였다']



# 2-1. Bag of Words (Bow)

In [7]:
def bulid_vocab(tokenized_corpus):
  word2idx = dict()

  idx = 0
  for sent_i in tokenized_corpus:
    for word_i in sent_i:
      try:
        tmp = word2idx[word_i]
      except:
        word2idx[word_i] = idx
        idx += 1

  return word2idx
  
nsmc5_word2idx = bulid_vocab(tokenized_nsmc5_corpus)
print(len(nsmc5_word2idx))
print(nsmc5_word2idx)


20
{'사이': 0, '몬페': 1, '그': 2, '의': 3, '익살스런': 4, '연기': 5, '가': 6, '돋보였던': 7, '영화': 8, '!': 9, '스파이더맨': 10, '에서': 11, '늙어': 12, '보이기만': 13, '했던': 14, '커스틴': 15, '던스트': 16, '너무나도': 17, '이뻐': 18, '보였다': 19}


In [8]:
def bow(word_list, word2idx):
  bow_vector = [0] * len(word2idx)

  for word_i in word_list:
    try:
      bow_vector[word2idx[word_i]] += 1
    except:
      pass

  return bow_vector

print(tokenized_nsmc5_corpus[3])
print(bow(tokenized_nsmc5_corpus[3], nsmc5_word2idx))

['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']
[1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


# 2-1-1. Sentiment Analysis (Bow)

In [9]:
import numpy as np
from tqdm import tqdm

#1. 각 단어의 긍정 수치를 계산(각 단어별 긍정 문장 수 / 단어가 등장한 문장 수)
#2. 한 문장의 긍정 수치는 그 문장을 구성하는 단어의 긍정 수치의 평균
#3. t 값은 전체 corpus의 긍정 / 부정 문장의 비율로 결정
#4. 긍정 수치가 t 이상 => 긍정, t 미만 => 부정

# nsmc 데이터 중 앞의 1000개
sents = [dataset["train"][idx]["document"] for idx in range(1000)]
labels =[dataset["train"][idx]["label"] for idx in range(1000)]

tokenized_nsmc1000_corpus = []

#1000개의 문장에 대해서 tokenized
for sent_i in tqdm(sents): # for _ in tqdm(range(10))
  new_sent_i = spell_checker.check(sent_i).checked
  new_sent_i = spacing(new_sent_i)
  words_i = [w for w, p in okt.pos(new_sent_i)]

  tokenized_nsmc1000_corpus.append(words_i)

# vocab 생성
nsmc1000_word2idx = bulid_vocab(tokenized_nsmc1000_corpus)
nsmc1000_idx2word = {k: v for v, k in nsmc1000_word2idx.items()}
print("전체 단어: {0}".format(len(nsmc1000_word2idx)))
print()

100%|██████████| 1000/1000 [03:19<00:00,  5.02it/s]

전체 단어: 4443



In [10]:
#vocab에 등장하는 단어별 긍정 수치 계산
pos_score = np.zeros(len(nsmc1000_word2idx)) # 긍정 수치 numct.array
counts = np.zeros(len(nsmc1000_word2idx)) # 각 단어의 등장 횟수
num_pos = 0
num_token = 0

for idx, word_list in enumerate(tokenized_nsmc1000_corpus):
  label_i = labels[idx] # 각 문장의 긍정(1), 부정(0), label

  for word_i in word_list:
    idx_i = nsmc1000_word2idx[word_i]
    counts[idx_i] += 1 # 각 단어의 등장 횟수 +1
    pos_score[idx_i] += label_i # 각 단어가 긍정인 경우에 + 1
  num_pos += label_i + len(word_list) # 긍정 문장인 경우 +1
  num_token += len(word_list)
  
pos_score = pos_score / counts # 원소별로 나누기 연산이 진행됨

t = num_pos / num_token

print("t 값: {0}".format(t))
print()


t 값: 1.0321295631163063



In [11]:
print(pos_score)

[0.42424242 0.         0.40555556 ... 1.         1.         1.        ]


In [12]:
#Bow 기반 5개 문장의 긍정, 부정, 수치 계산 및 예측
for idx, word_list in enumerate(tokenized_nsmc1000_corpus[:5]):
  bow_vector_i = np.array(bow(word_list, nsmc1000_word2idx))
  weighted_bow_vector = bow_vector_i * pos_score
  sent_score = np.sum(weighted_bow_vector) / np.sum(bow_vector_i)
  
  result_i = "긍정" if sent_score >= t else "부정"
  answer_i = "긍정" if labels[idx] == 1 else "부정"

  print("{0}번째 문장 : {1}".format(idx, sents[idx]))
  print("정답: {0} | 예측: {1} | 긍정수치: {2}".format(answer_i, result_i, sent_score))
  print()

0번째 문장 : 아 더빙.. 진짜 짜증나네요 목소리
정답: 부정 | 예측: 부정 | 긍정수치: 0.185038732225433

1번째 문장 : 흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
정답: 긍정 | 예측: 부정 | 긍정수치: 0.6129973887174245

2번째 문장 : 너무재밓었다그래서보는것을추천한다
정답: 부정 | 예측: 부정 | 긍정수치: 0.40889652815277666

3번째 문장 : 교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
정답: 부정 | 예측: 부정 | 긍정수치: 0.27780186230688386

4번째 문장 : 사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다
정답: 긍정 | 예측: 부정 | 긍정수치: 0.8045686931863756



In [13]:
# Bow 기반 전체 1000개 문장에 대한 문장의 긍정, 부정 예측 평가
pred = list()
for idx, word_list in enumerate(tokenized_nsmc1000_corpus):
  bow_vector_i = np.array(bow(word_list, nsmc1000_word2idx))
  weighted_bow_vector = bow_vector_i + pos_score
  sent_score = np.sum(weighted_bow_vector) / np.sum(bow_vector_i)

  if sent_score >= t:
    pred_i = 1
  else:
    pred_i = 0

  pred.append(pred_i)

pred = np.array(pred)
labels = np.array(labels)

print("정확도: {0}".format(np.sum(pred==labels)/len(pred)))

정확도: 0.492


# 2-1-2. Sentence Simillarty (Bow)

In [14]:
bow_vector_0 = np.array(bow(tokenized_nsmc1000_corpus[0], nsmc1000_word2idx))

cos_sim = []

for idx, word_list in enumerate(tokenized_nsmc1000_corpus[1:]):
  bow_vector_i = np.array(bow(word_list, nsmc1000_word2idx))

  cos_sim_i = np.dot(bow_vector_0, bow_vector_i)/np.linalg.norm(bow_vector_0)*np.linalg.norm(bow_vector_i)
  cos_sim.append(cos_sim_i)

cos_sim = np.array(cos_sim)

#[100, 300, 400, 200].argsort() =. [0, 3, 1, 2] (크기 순으로 정렬했을 때 각 원소의 index가 변환됨)
top_5_idx = cos_sim.argsort()[-5:][::-1]

print("Query: {0}".format(sents[0]))
print("--------------- TOP5 ----------------")
for idx in top_5_idx:
  print(sents[idx+i])

Query: 아 더빙.. 진짜 짜증나네요 목소리
--------------- TOP5 ----------------
어떻게 이런 상상을 대단하다.
야 세르게이!!! 작은 고추의 매운맛을 보여주마
참 재밌게 봤는데... 귀여운 지미 +_+
평점 조절이 필요하다 라는 생각이 든다!
평생 기억할만한 영화,정상적인 소재는 아니지만


# 2-2. TF-IDF (Term Frequency - Inverse Document Frequency)

문자 내 각 단어의 빈도를 고려, 단어마다 가중치를 두는 방법

흔히 등장하는 단어 = 낮은 가중치

In [15]:
def calculate_idf(word2idx, tokenized_corpus):
  df = np.zeros(len(word2idx))

  for sent_i in tokenized_corpus:
    sent_i_df = np.zeros(len(word2idx))

    for word_i in sent_i:
      sent_i_df[word2idx[word_i]] = 1

    df += sent_i_df
  idf = np.log(len(tokenized_corpus) / (1 + df))

# 10000개의 문서에서 딱 1번 존재하는 문서

  return idf

idf_nsmc5_corpus = calculate_idf(nsmc5_word2idx, tokenized_nsmc5_corpus)
print(idf_nsmc5_corpus)

[-0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156
 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156
 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156
 -0.18232156 -0.18232156]


In [18]:
def tf_idf(word_list, word2idx, idf):
  tf_vector = [0] * len(word2idx)

  for word_i in word_list:
    try:
      tf_vector[word2idx[word_i]] += 1
    except:
      pass
  
  tf_idf_vector = tf_vector * idf

  return tf_idf_vector
print(tokenized_nsmc5_corpus[0])
print(tf_idf(tokenized_nsmc5_corpus[0], nsmc5_word2idx, idf_nsmc5_corpus))

['사이', '몬페', '그', '의', '익살스런', '연기', '가', '돋보였던', '영화', '!', '스파이더맨', '에서', '늙어', '보이기만', '했던', '커스틴', '던스트', '가', '너무나도', '이뻐', '보였다']
[-0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156
 -0.36464311 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156
 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156 -0.18232156
 -0.18232156 -0.18232156]


# 2-2-1. Sentiment Analysis (TF-IDF)